# ImmoEliza Project - Part 3: Regression 

# Step 1: Data Cleaning

In [1]:
# Import libraries
import pandas as pd

In [2]:
# Load cleaned dataset
cleaned_data = pd.read_csv("./data/cleaned-data.csv")

In [3]:
cleaned_data.columns

Index(['zip_code', 'commune', 'province', 'type_of_property',
       'subtype_of_property', 'price', 'building_condition', 'facade_number',
       'living_area', 'equipped_kitchen', 'bedroom_nr', 'swimming_pool',
       'furnished', 'open_fire', 'terrace', 'garden', 'plot_surface',
       'sub_property_group_encoded'],
      dtype='object')

In [4]:
# Load raw dataset
raw_data = pd.read_csv("./data/raw_data.csv")

In [5]:
raw_data.columns

Index(['zip_code', 'commune', 'province', 'type_of_property',
       'subtype_of_property', 'price', 'building_condition', 'facade_number',
       'living_area', 'equipped_kitchen', 'bedroom_nr', 'swimming_pool',
       'furnished', 'open_fire', 'terrace', 'garden', 'plot_surface'],
      dtype='object')

In [6]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26147 entries, 0 to 26146
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   zip_code             26147 non-null  int64  
 1   commune              26147 non-null  object 
 2   province             26147 non-null  object 
 3   type_of_property     26147 non-null  int64  
 4   subtype_of_property  26147 non-null  object 
 5   price                26111 non-null  float64
 6   building_condition   19457 non-null  object 
 7   facade_number        16785 non-null  float64
 8   living_area          26147 non-null  int64  
 9   equipped_kitchen     26147 non-null  object 
 10  bedroom_nr           26147 non-null  int64  
 11  swimming_pool        26147 non-null  int64  
 12  furnished            26147 non-null  int64  
 13  open_fire            26147 non-null  int64  
 14  terrace              26147 non-null  int64  
 15  garden               26147 non-null 

## 1. Handle Duplicates and Missing Values

### 1.1 Duplicates

In [7]:
# Check for duplicates and count them
num_duplicates = raw_data.duplicated().sum()
num_duplicates

np.int64(1284)

Note on duplicates: During scraping, duplicates were eliminated based on property id, thus the 1284 entries identified as duplicates just have the same features which is often the case with new real estate projects, but are in fact individual properties.

### 1.2 Blank Space

In [8]:
# Check for leading/trailing spaces in string columns

columns_with_spaces = [col for col in raw_data.columns if raw_data[col].dtype == 'object' and raw_data[col].str.contains(r'^\s+|\s+$').any()]
columns_with_spaces

[]

In [9]:
# Check for multiple spaces within a string
columns_with_spaces = [col for col in raw_data.columns if raw_data[col].dtype == 'object' and raw_data[col].str.contains(r'  ').any()]
columns_with_spaces

[]

### 1.3 Missing Values

In [10]:
# See summary of missing values
raw_data.isna().sum()

zip_code                  0
commune                   0
province                  0
type_of_property          0
subtype_of_property       0
price                    36
building_condition     6690
facade_number          9362
living_area               0
equipped_kitchen          0
bedroom_nr                0
swimming_pool             0
furnished                 0
open_fire                 0
terrace                   0
garden                    0
plot_surface              0
dtype: int64

In [11]:
# Dropping entries with missing values in "price" (36)
dfc = raw_data.dropna(subset=['price'], how='all')
dfc.reset_index(drop=True, inplace=True)                              # Added "drop=True" to drop the old index

In [12]:
# Handling missing values in "facade_number" (9362)

# Compute median facade number by subtype
facade_dict = dfc.groupby("subtype_of_property")["facade_number"].median().to_dict()

# Impute values
dfc.loc[:, "facade_number"] = dfc["facade_number"].fillna(dfc["subtype_of_property"].map(facade_dict))

In [13]:
# Check new length of dataset
dfc.shape

(26111, 17)

In [14]:
# Check unique values in "subtype of property"
unique_subtypes = dfc["subtype_of_property"].unique()
unique_subtypes

array(['apartment', 'penthouse', 'flat studio', 'ground floor', 'duplex',
       'loft', 'service flat', 'kot', 'triplex', 'apartment unit',
       'duplex unit', 'ground floor unit', 'kot unit', 'flat studio unit',
       'penthouse unit', 'loft unit', 'triplex unit', 'service flat unit',
       'house', 'villa', 'apartment block', 'mansion',
       'exceptional property', 'mixed use building', 'country cottage',
       'town house', 'castle', 'bungalow', 'manor house', 'farmhouse',
       'chalet', 'other property', 'house unit', 'villa unit',
       'town house unit', 'mixed use building unit'], dtype=object)

In [15]:

# Create function to remove 'unit' form sub type of property, so 'apartment' and 'apartment unit' become one category and so on
def remove_unit(subtype_of_property):
    if 'unit' in subtype_of_property:
        subtype_of_property = subtype_of_property.replace(' unit', '')
    return subtype_of_property

# Apply the function to remove 'unit' from the sub type col:
dfc.loc[:, 'subtype_of_property'] = dfc['subtype_of_property'].apply(remove_unit)

In [34]:
# Check unique values in "subtype of property"
unique_subtypes = dfc["subtype_of_property"].unique()
unique_subtypes

array(['apartment', 'penthouse', 'flat studio', 'ground floor', 'duplex',
       'loft', 'service flat', 'kot', 'triplex', 'house', 'villa',
       'apartment block', 'mansion', 'exceptional property',
       'mixed use building', 'country cottage', 'town house', 'castle',
       'bungalow', 'manor house', 'farmhouse', 'chalet', 'other property'],
      dtype=object)

In [16]:
# Check unique values in "building_condition"
unique_conditions = dfc["building_condition"].unique()
unique_conditions

array(['good', 'as new', 'just renovated', 'to be done up', 'to renovate',
       nan, 'to restore'], dtype=object)

In [17]:
# Replace NaN in "building_condition" with 'Unknown', since it cannot be guessed based on other criteria
dfc.loc[:, "building_condition"] = dfc["building_condition"].fillna("Unknown")

In [18]:
# See summary of missing values
dfc.isna().sum()

zip_code               0
commune                0
province               0
type_of_property       0
subtype_of_property    0
price                  0
building_condition     0
facade_number          0
living_area            0
equipped_kitchen       0
bedroom_nr             0
swimming_pool          0
furnished              0
open_fire              0
terrace                0
garden                 0
plot_surface           0
dtype: int64

## 2. Handle Categorical/ Text Data

### 2.1 Inspect Categorical Data

In [19]:
# Check columns with categorical / text data

# Count quantitative (numerical) variables
quantitative_vars = dfc.select_dtypes(include=['int64', 'float64']).columns
num_quantitative = len(quantitative_vars)

# Count qualitative (categorical) variables
qualitative_vars = dfc.select_dtypes(include=['object', 'category']).columns.to_list()
num_qualitative = len(qualitative_vars)

print(f"Quantitative variables: {num_quantitative}")
print(f"Qualitative variables: {num_qualitative}")

print(f"Qualitative variables: {qualitative_vars}")

Quantitative variables: 12
Qualitative variables: 5
Qualitative variables: ['commune', 'province', 'subtype_of_property', 'building_condition', 'equipped_kitchen']


In [ ]:
# Check unique values

unique_communes = dfc["commune"].unique()
print("Number of unique communes:", len(unique_communes))

Number of unique communes: 864


array(['Berchem', 'Deurne', 'Hoboken', 'Uccle', 'Antwerpen', 'Ixelles',
       'Wommelgem', 'Boom', 'Kortrijk', 'Anderlecht', 'Zeveren',
       'Zeveneken', 'Willebroek', 'Waregem', 'Wilrijk', 'Sleidinge',
       'Gent', 'Stabroek', 'Molenbeek-Saint-Jean', 'Koekelberg',
       'Berchem-Sainte-Agathe', 'Aalst', 'Bruxelles', 'Sint-Niklaas',
       'Westende', 'Zandvoorde', 'Houthalen', 'Middelkerke', 'Ledeberg',
       'Walem', 'Schaerbeek', 'Woluwe-Saint-Lambert', 'Zottegem',
       'Westkapelle', 'Rocourt', 'Wenduine', 'Rumbeke', 'Uitkerke',
       'Etterbeek', 'Waarloos', 'Borgerhout', 'Leuven', 'Auderghem',
       'Merksem', 'Ganshoren', 'Avelgem', 'Harelbeke', 'Vissenaken',
       'Sint-Denijs-Westrem', 'Sint-Lambrechts-Herk', 'Uitbergen',
       'Laeken', 'Sint-Pieters-Leeuw', 'Orp-le-Grand', 'Zétrud-Lumay',
       'Waardamme', 'Sint-Joris', 'Woumen', 'Rupelmonde', 'Soignies',
       'Heldergem', 'Saint-Josse-ten-Noode', 'Sint-Gillis-Dendermonde',
       'Ramsdonk', 'Weert', 'Welde

In [23]:
unique_provinces = dfc["province"].unique()
unique_provinces

array(['Antwerpen', 'Bruxelles', 'West-Vlaanderen', 'Oost-Vlaanderen',
       'Limburg', 'Liège', 'Vlaams Brabant', 'Brabant Wallon', 'Hainaut',
       'Luxembourg', 'Namur'], dtype=object)

In [25]:
unique_zipcodes = dfc["zip_code"].unique()
print("Number of unique zip codes:", len(unique_zipcodes))

Number of unique zip codes: 873


In [26]:
unique_subtypes = dfc["subtype_of_property"].unique()
unique_subtypes

array(['apartment', 'penthouse', 'flat studio', 'ground floor', 'duplex',
       'loft', 'service flat', 'kot', 'triplex', 'house', 'villa',
       'apartment block', 'mansion', 'exceptional property',
       'mixed use building', 'country cottage', 'town house', 'castle',
       'bungalow', 'manor house', 'farmhouse', 'chalet', 'other property'],
      dtype=object)

In [27]:
unique_conditions

array(['good', 'as new', 'just renovated', 'to be done up', 'to renovate',
       nan, 'to restore'], dtype=object)

In [28]:
unique_kitchens = dfc["equipped_kitchen"].unique()
unique_kitchens

array(['installed', '0', 'hyper equipped', 'semi equipped',
       'not installed', 'usa semi equipped', 'usa hyper equipped',
       'usa installed', 'usa uninstalled'], dtype=object)

### 2.2 Encode Categorical Data

#### 2.2.1 Kitchen Equipment

## 3. Select Features 
(Remove features that have a too strong correlation between them)